In [5]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests
import shutil


In [71]:
class SECSubmissions:
    def __init__(self,ticker):
        self.ticker = ticker 
        self.CIK = self.getCIK()
        self.filings = self.getSECFilings(self.CIK)
        self.filings = self.findDate(self.filings)
        self.fiscalYear = self.findFiscalYear(self.filings)
        self.filings = self.mergeFiscalYearWithFilings(self.filings, self.fiscalYear)
        self.filings = self.createSubmissionName(self.filings)
        
    def getCIK(self): #get SEC code for company based on ticker
        headers={"User-Agent": "Mozilla/5.0"}
        symbol_to_cik = requests.get("https://www.sec.gov/files/company_tickers.json").json() #returns a json dictionary with a indexed list of all different companies 
        ciks = {info["ticker"]:info["cik_str"] for key,info in symbol_to_cik.items()} #create dictionary indexable by ticker
        return ciks[self.ticker]
    
        
    #input: CIK
    #request the data on every submission to SEC website for the company
    def getSECFilings(self,CIK):
        headers={"User-Agent": "Mozilla/5.0"}
        edgar_filings = requests.get(f"https://data.sec.gov/submissions/CIK{CIK:0>10}.json", headers=headers).json()
        filings = pd.DataFrame(edgar_filings["filings"]["recent"])
        filings = filings.loc[filings["reportDate"]>"2014-01-01"]#filings pre-2014 are unable to be downloaded due to different excel format
        filings = filings.loc[filings["form"].isin(["10-Q","10-K"])].reset_index(drop=True) #drop all filings that are not 10Q or 10K
        return filings 
    
    #input filings
    #output filings with mo and year
    def findDate(self,filings):
        filings["Month"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%m").astype("int") #get Month of all filings 
        filings["Year"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%Y").astype("int") #get Year of all filings
        return filings
    
    #input a list of 10k filings with mo and year
    #find one fiscal year of data via 10k, then find all possible months filed and organize to fiscal period
    def findFiscalYear(self,filings):
        #get a fiscal Year of data by finding the last 10K and the three 10Qs before it
        TenKIndex = filings.loc[filings["form"]=="10-K"].index[0] 
        fiscalYearKey = filings.loc[TenKIndex:TenKIndex+3,["Month","form"]].copy().reset_index(drop=True)
        fiscalYearKey["Period"] = ["Year Ended", "Q3","Q2","Q1"]
        
        #fiscal year data for certain period can vary between months ie. both Jan & Feb, therefore I found all possible months of filings and found appropriate period for each month
        fiscalYear = pd.DataFrame({"Month":filings.Month.unique()})
        fiscalYear.index = list(fiscalYear.Month.apply(lambda x: np.argmin(np.abs(fiscalYearKey.Month-x)))) #using fiscal year key, I found the index of the period in fiscalYearKey with closest month to each month 
        fiscalYear = pd.merge(fiscalYear,fiscalYearKey, left_index=True, right_index=True, suffixes=("","_x")) #I merged together together all of the possible months with their corresponding period based on the key
        fiscalYear = fiscalYear.drop(columns="Month_x")
        
        return fiscalYear
    
    #input filings of 10k/q
    #merge by month to find which fiscal period, if filing of 10q is later in year than 10k, then adjust year so it is a part of right fiscal year
    def mergeFiscalYearWithFilings(self, filings, fiscalYear):
        print(fiscalYear)
        #merge fiscal year labels with filings by the month that they were reported
        filings = pd.merge(filings,fiscalYear,on=["Month","form"],how="left")    
        #set Fiscal Year
        filings["Fiscal Year"] = filings["Year"]
        #get Quarters that are a part of different fiscal Year than report date
        TenKMo = fiscalYear.loc[fiscalYear["form"]=="10-K"]["Month"].iloc[0]
        filings.loc[(filings["Month"]>TenKMo)&(filings["form"]!="10-K"),"Fiscal Year"] += 1 #the 10K is reported in a month before the 10Q so therefore the 10Q is the next fiscal year 
        
        return filings
    
    

    #input filings
    #take the fiscal period and year and create a string, if different fiscal Year than filing date use parenthesis
    def createSubmissionName(self,filings):
        filings["Name"] = filings.apply(self.submissionName,axis=1)
        return filings
        
    def submissionName(self,row):
        period = row["Period"]
        fiscalYear = row["Fiscal Year"]
        Year = row["Year"]
        if fiscalYear != Year:
                name = f"{period} {fiscalYear} ({Year})"
        else: 
                name = f"{period} {fiscalYear}"
        return name
    
    
    
        

In [72]:
class WriteSECFinancialStatements(SECSubmissions):
    def __init__(self,ticker):
        super().__init__(ticker)
        self.path =  os.path.join("../input",f"Financial Statement {self.ticker}")
        self.folder = os.path.join(self.path,"raw")
        
        self.createFolder()
        self.writeFilings(self.filings)
        
        self.fiscalYear.to_csv(os.path.join(self.path,"fiscalYear.csv"))
    
    #create a folder to put all of the filings in
    def createFolder(self):
        if f"Financial Statement {self.ticker}" not in os.listdir("../input"): os.mkdir(self.path)
        if "raw" in os.listdir(self.path): shutil.rmtree(self.folder)
        os.mkdir(self.folder)
    
    #input: filings
    #iterate through and use the accession number to grab filings from SEC, compile into excel spreadsheet
    def writeFilings(self,filings):
        for i,row in self.filings.iterrows():
            name = row["Name"]
            accessionNum = row["accessionNumber"].replace("-","")
            
            url = f"https://www.sec.gov/Archives/edgar/data/{self.CIK}/{accessionNum}/Financial_Report.xlsx"
            req = requests.get(url,headers={"User-Agent": "Mozilla/5.0"})

            file = open(os.path.join(self.folder,name+".xlsx"),"wb")
            file.write(req.content)
            
        

In [73]:
WriteSECFinancialStatements("AAPL")

   Month  form      Period
0      9  10-K  Year Ended
1      6  10-Q          Q3
1      7  10-Q          Q3
2      3  10-Q          Q2
2      4  10-Q          Q2
3     12  10-Q          Q1


In [74]:
class FinancialReport():
    def __init__(self,filePath):
        self.filePath = filePath
        self.readFile()

    def readFile(self):
        self.statements = pd.read_excel(self.filePath,sheet_name=None)
    
    def parseSheet(self, sheet):
        sheet.columns = ["label"]+list(sheet.columns[1:])
        
        sheet = sheet.loc[~sheet.label.isnull(), sheet.columns.notna()]
        
        #remove rows where the label is just [1] or [2] because they are empty
        sheet = sheet.loc[sheet.label.str.contains("[a-zA-Z]",regex=True)]
       
        if not sheet.iloc[:,1].astype("string").str.isnumeric().any():
            sheet=sheet.drop(sheet.columns[1],axis=1) #delete columns where there is only one value ie. a note like [1]
        
        sheet = sheet.iloc[:,:2] 
        return sheet
        
    def findBalanceSheet(self):
        #find sheet where the first sheet value (header) includes balance
        balanceSheets = [sheet for name,sheet in self.statements.items() if "balance" in str(sheet.columns[0]).lower()]
        balance = balanceSheets[0]
        
        balance = self.parseSheet(balance)
        
        return balance
        
    def findIncomeSheet(self):
        #find sheet where the first sheet value (header) includes income
        incomeSheets = [sheet for name,sheet in self.statements.items() if "income" in str(sheet.columns[0]).lower() or "operations" in str(sheet.columns[0]).lower()]
        income = incomeSheets[0]
        
        #remove the first row which doesn't say the date but instead says the length of time period ie. 3 mo ended 
        income.columns = income.iloc[0]
        income = income.drop(0)
        
        income = self.parseSheet(income)
        return income

        
    def findCashflowSheet(self): 
        #find sheet where the first sheet value (header) includes income
        cashSheets = [sheet for name,sheet in self.statements.items() if "cash" in str(sheet.columns[0]).lower()]
        cashflow = cashSheets[0]
        
        #remove the first row which doesn't say the date but instead says the length of time period ie. 3 mo ended 
        cashflow.columns = cashflow.iloc[0]
        cashflow = cashflow.drop(0)
        
        cashflow = self.parseSheet(cashflow)
        return cashflow
        
        

        
   
    

In [75]:
x = FinancialReport("../input/Financial Statement XOM/raw/Year Ended 2017.xlsx")
x.findBalanceSheet()

,label,"Dec. 31, 2017"
0,Current assets,NaN
1,Cash and cash equivalents,3177
2,"Notes and accounts receivable, less estimated ...",25597
3,Inventories,NaN
4,"Crude oil, products and merchandise",12871
5,Materials and supplies,4121
6,Other current assets,1368
7,Total current assets,47134
8,"Investments, advances and long-term receivables",39160
9,"Property, plant and equipment, at cost, less a...",252630


In [76]:
class compileFilings:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker 
        self.fromDate = fromDate
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.folder = os.path.join(self.path,"raw")
        
        self.filings = self.getFilings()
        self.filings = self.getFilingsPostDate(self.filings)
        self.files = self.getFilePathsFromFilings(self.filings)
        
        
        self.writeIncomeExcel()
        self.writeCashflowExcel()
        self.writeBalanceExcel()
    
    def getFilings(self): #get the names of all of the filings
        #get a list of all of the filings 
        filings = os.listdir(self.folder)
        if ".DS_Store" in filings:
            filings.remove(".DS_Store")
        filings = [filing for filing in filings if"~" not in filing]
        
        return filings
    
    def getYears(self,quarters): #go through each file name and find the year of filing
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    
    
    def getPossibleFilingsFromYears(self, years):
        return [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"]]
    
    def getFilingsPostDate(self,filings):
        years = range(self.fromDate,2023)
        names = self.getPossibleFilingsFromYears(years)
        names = ["Year Ended "+str(self.fromDate-1)]+names
        files = [filing for name in names for filing in filings if name in filing] 
        return files
    
    def getFilePathsFromFilings(self,filings): #take file names and return a list of the paths to the file
        return [os.path.join(self.folder,filing) for filing in filings]
        
    def writeIncomeExcel(self):
        
        file = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        Income = pd.ExcelWriter(file)
        
        #for each filing, read the filing into a dataframe and save to a combined excel file
        for file in self.files:
            print(file)
            statement = FinancialReport(file)
            income = statement.findIncomeSheet()
            name = file.split("/")[-1].split(".")[0]
            income.to_excel(Income, index=False, sheet_name=name)
        
        Income.save()
        
    def writeCashflowExcel(self):
        
        file = os.path.join(self.path, f"Cashflow Statements All-{self.ticker}.xlsx")
        Cashflow = pd.ExcelWriter(file)
        #for each filing, read the filing into a dataframe and save to a combined excel file
        for file in self.files:
            statement = FinancialReport(file)
            cashflow = statement.findCashflowSheet()
            name = file.split("/")[-1].split(".")[0]
            cashflow.to_excel(Cashflow, index=False, sheet_name=name)
        
        Cashflow.save()
        
    
    def writeBalanceExcel(self):
        
        file = os.path.join(self.path, f"Balance Statements All-{self.ticker}.xlsx")
        Balance = pd.ExcelWriter(file)
        #for each filing, read the filing into a dataframe and save to a combined excel file
        for file in self.files:
            statement = FinancialReport(file)
            balance = statement.findBalanceSheet()
            name = file.split("/")[-1].split(".")[0]
            balance.to_excel(Balance, index=False, sheet_name=name)
        
        Balance.save()
        
        

In [77]:
WriteSECFinancialStatements("AAPL")
compileFilings("AAPL",2017)

   Month  form      Period
0      9  10-K  Year Ended
1      6  10-Q          Q3
1      7  10-Q          Q3
2      3  10-Q          Q2
2      4  10-Q          Q2
3     12  10-Q          Q1
../input/Financial Statement AAPL/raw/Year Ended 2016.xlsx
../input/Financial Statement AAPL/raw/Q1 2017 (2016).xlsx
../input/Financial Statement AAPL/raw/Q2 2017.xlsx
../input/Financial Statement AAPL/raw/Q3 2017.xlsx
../input/Financial Statement AAPL/raw/Year Ended 2017.xlsx
../input/Financial Statement AAPL/raw/Q1 2018 (2017).xlsx
../input/Financial Statement AAPL/raw/Q2 2018.xlsx
../input/Financial Statement AAPL/raw/Q3 2018.xlsx
../input/Financial Statement AAPL/raw/Year Ended 2018.xlsx
../input/Financial Statement AAPL/raw/Q1 2019 (2018).xlsx
../input/Financial Statement AAPL/raw/Q2 2019.xlsx
../input/Financial Statement AAPL/raw/Q3 2019.xlsx
../input/Financial Statement AAPL/raw/Year Ended 2019.xlsx
../input/Financial Statement AAPL/raw/Q1 2020 (2019).xlsx
../input/Financial Statement AAPL/raw